# 2-2: MLflow Model Registry

## 학습 목표
- MLflow Model Registry 기본 개념 이해
- 모델 등록 및 버전 관리
- **Alias 방식 (최신)** vs Stage 방식 (레거시) 비교
- Champion/Challenger 패턴으로 모델 배포 관리

## 이 노트북에서 할 것
```
2-1의 best_model_final → Model Registry 등록
                         ↘ 버전 관리
                         ↘ Alias 설정 (champion)
                         ↘ 모델 로드 및 추론
```

## 사전 요구사항
- 2-1 노트북 실행 완료
- `best_model_final` run 존재

## 예상 시간: 약 2시간

In [1]:
# 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# ML 패키지
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, 
    recall_score, precision_score, confusion_matrix
)

# MLflow
import mlflow
from mlflow.tracking import MlflowClient

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 경로
DATA_PROCESSED = Path('../../data/processed')

print(f"MLflow 버전: {mlflow.__version__}")
print("패키지 로드 완료!")

MLflow 버전: 3.8.1
패키지 로드 완료!


---

## Part 1: Model Registry 개념

### 1-1. Model Registry란?

**Model Registry** = 모델 버전 관리 + 배포 관리 시스템

```
MLflow 4대 컴포넌트:
1. Tracking: 실험 추적 ← 2-1에서 완료
2. Projects: 재현 가능한 패키징
3. Models: 모델 배포 표준화
4. Registry: 모델 버전 관리 ← 이번 노트북 ⭐
```

### 구조

```
Model Registry
└── fraud_detector (Registered Model)
    ├── Version 1: AUC 0.88 (2024-01-01)
    ├── Version 2: AUC 0.90 (2024-01-15) ← @champion
    └── Version 3: AUC 0.91 (2024-02-01) ← @challenger
```

### 왜 필요한가?

**기존 방식의 문제점:**
```python
# 모델 파일 직접 관리
models/
├── xgb_model_v1.joblib
├── xgb_model_v2.joblib
├── xgb_model_v2_fixed.joblib  # ???
├── xgb_model_final.joblib
└── xgb_model_final_final.joblib  # 혼란!
```

**Model Registry 사용 시:**
```python
# 버전 자동 관리
models:/fraud_detector/1  # Version 1
models:/fraud_detector/2  # Version 2
models:/fraud_detector@champion  # 프로덕션 모델
models:/fraud_detector@challenger  # 테스트 중인 모델
```

### 1-2. FDS에서 왜 필요한가?

**금융권 FDS의 모델 관리 요구사항:**

| 요구사항 | Model Registry 해결책 |
|----------|----------------------|
| 모델 버전 추적 | 자동 버전 관리 (v1, v2, ...) |
| 롤백 필요 | 이전 버전으로 즉시 alias 변경 |
| A/B 테스트 | champion vs challenger 비교 |
| 감사 추적 | 누가, 언제, 어떤 모델 배포했는지 기록 |
| 규제 준수 | 모델 변경 이력 보존 |

**실제 시나리오:**
```
1. 새 모델 학습 → Version 3 등록
2. 오프라인 테스트 → validation_status: "testing"
3. A/B 테스트 → challenger alias 설정
4. 성능 검증 완료 → champion alias로 승격
5. 문제 발생 → 이전 버전으로 champion 롤백
```

### 1-3. Stage vs Alias 방식 비교

| 구분 | Stage 방식 (레거시) | Alias 방식 (MLflow 2.9+ 권장) |
|------|---------------------|------------------------------|
| 상태 | **Deprecated** | **권장** ⭐ |
| 방식 | None → Staging → Production | @champion, @challenger |
| 유연성 | 고정된 3단계만 가능 | 자유롭게 alias 생성 |
| 동시 사용 | 한 stage에 하나만 | 여러 alias 동시 적용 가능 |
| API | `transition_model_version_stage()` | `set_registered_model_alias()` |
| URI | `models:/name/Production` | `models:/name@champion` |

**MLflow 공식 권장:**
> "Model version aliases provide a flexible way to create named references 
> for particular model versions. A common pattern is to use aliases like 
> `champion` for a currently deployed production model and `challenger` 
> for a new model being evaluated."

**이 노트북에서는:**
- **Part 3**: Alias 방식 (최신) - 메인으로 학습
- **Part 4**: Stage 방식 (레거시) - 참고용으로 간단히 다룸

In [2]:
# 예제: MlflowClient 연결

# Tracking URI 설정 (2-1과 동일)
mlflow.set_tracking_uri("file:./mlruns")

# 실험 선택
mlflow.set_experiment("fds-xgboost")

# MlflowClient 생성 - Registry 작업에 필요
client = MlflowClient()

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"MlflowClient 생성 완료!")

Tracking URI: file:./mlruns
MlflowClient 생성 완료!


In [ ]:
# 실습 1: MlflowClient 연결 확인

# TODO 1: Tracking URI 설정
mlflow.set_tracking_uri("_____")

# TODO 2: MlflowClient 생성
client = _____

# 확인
print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Client type: {type(client).__name__}")

In [3]:
# 실습 1 정답

# Tracking URI 설정
mlflow.set_tracking_uri("file:./mlruns")

# MlflowClient 생성
client = MlflowClient()

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Client type: {type(client).__name__}")

Tracking URI: file:./mlruns
Client type: MlflowClient


In [4]:
# 체크포인트 1: MlflowClient 연결 확인
assert client is not None, "MlflowClient 생성 실패!"
assert mlflow.get_tracking_uri() == "file:./mlruns", "Tracking URI 설정 오류!"
print("체크포인트 1 통과: MlflowClient 연결 성공!")

체크포인트 1 통과: MlflowClient 연결 성공!


---

## Part 2: 모델 등록

### 2-1. 2-1의 best_model_final 찾기

2-1에서 학습한 `best_model_final` run을 찾아서 Model Registry에 등록합니다.

In [5]:
# 예제: best_model_final run 찾기 (아티팩트 있는 run)

# fds-xgboost 실험에서 best_model_final 검색
experiment = mlflow.get_experiment_by_name("fds-xgboost")

if experiment is None:
    print("❌ fds-xgboost 실험을 찾을 수 없습니다!")
    print("2-1 노트북을 먼저 실행하세요.")
else:
    # best_model_final로 태그된 run 전체 검색
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        filter_string="tags.mlflow.runName = 'best_model_final'",
        order_by=["start_time DESC"]
    )

    if runs:
        # 아티팩트가 있는 run 찾기
        best_run = None
        for run in runs:
            artifacts = client.list_artifacts(run.info.run_id)
            artifact_names = [a.path for a in artifacts]
            if 'best_model' in artifact_names or 'model' in artifact_names:
                best_run = run
                break
        
        if best_run is None:
            print("⚠️ 아티팩트가 있는 best_model_final을 찾을 수 없습니다!")
            print("2-1 노트북을 다시 실행하세요.")
            # 첫 번째 run 사용 (fallback)
            best_run = runs[0]
        
        best_run_id = best_run.info.run_id
        
        # 아티팩트 경로 확인
        artifacts = client.list_artifacts(best_run_id)
        artifact_names = [a.path for a in artifacts]
        
        print(f"best_model_final Run 발견!")
        print(f"  Run ID: {best_run_id}")
        print(f"  Test AUC: {best_run.data.metrics.get('test_auc', 'N/A'):.4f}" if best_run.data.metrics.get('test_auc') else "  Test AUC: N/A")
        print(f"  Test AUPRC: {best_run.data.metrics.get('test_auprc', 'N/A'):.4f}" if best_run.data.metrics.get('test_auprc') else "  Test AUPRC: N/A")
        print(f"  Test Recall: {best_run.data.metrics.get('test_recall', 'N/A')}")
        print(f"  Artifacts: {artifact_names}")
    else:
        print("❌ best_model_final을 찾을 수 없습니다!")
        print("2-1 노트북을 먼저 실행하세요.")

⚠️ 아티팩트가 있는 best_model_final을 찾을 수 없습니다!
2-1 노트북을 다시 실행하세요.
best_model_final Run 발견!
  Run ID: f7c9439e4e384e2e8cac949255b76214
  Test AUC: 0.9026
  Test AUPRC: 0.5434
  Test Recall: N/A
  Artifacts: ['feature_importance.png']


### 2-2. 모델 등록 방법

**모델 등록 3가지 방법:**

```python
# 방법 1: log_model 시 바로 등록 (권장)
mlflow.sklearn.log_model(
    model, 
    name="model",
    registered_model_name="fraud_detector"  # ← 이 파라미터 추가
)

# 방법 2: 기존 run에서 등록
mlflow.register_model(
    model_uri=f"runs:/{run_id}/best_model",
    name="fraud_detector"
)

# 방법 3: MlflowClient로 등록
client.create_registered_model("fraud_detector")
client.create_model_version(
    name="fraud_detector",
    source=f"runs:/{run_id}/best_model",
    run_id=run_id
)
```

**우리는 방법 2 사용** (2-1에서 이미 모델을 로깅했으므로)

In [6]:
# 예제: 모델 등록

MODEL_NAME = "fraud_detector"

# 2-1의 best_model_final에서 모델 등록
model_uri = f"runs:/{best_run_id}/best_model"

# 모델 등록 (버전 자동 생성)
try:
    model_version = mlflow.register_model(
        model_uri=model_uri,
        name=MODEL_NAME
    )
    print(f"모델 등록 완료!")
    print(f"  이름: {model_version.name}")
    print(f"  버전: {model_version.version}")
    print(f"  소스: {model_version.source}")
    print(f"  Run ID: {model_version.run_id}")
except Exception as e:
    print(f"등록 실패: {e}")
    # 이미 등록된 경우 최신 버전 가져오기
    versions = list(client.search_model_versions(f"name='{MODEL_NAME}'"))
    if versions:
        model_version = versions[-1]
        print(f"\n기존 등록된 모델 사용: Version {model_version.version}")

Registered model 'fraud_detector' already exists. Creating a new version of this model...
2026/01/09 03:52:57 WARNING mlflow.tracking._model_registry.fluent: Run with id f7c9439e4e384e2e8cac949255b76214 has no artifacts at artifact path 'best_model', registering model based on models:/m-a0c19543686641aea3ab9b793e7813cd instead
Created version '2' of model 'fraud_detector'.

모델 등록 완료!
  이름: fraud_detector
  버전: 2
  소스: models:/m-a0c19543686641aea3ab9b793e7813cd
  Run ID: f7c9439e4e384e2e8cac949255b76214


In [7]:
# 등록된 모델 정보 확인

registered_model = client.get_registered_model(MODEL_NAME)

print(f"=== Registered Model: {registered_model.name} ===")
print(f"생성 시간: {registered_model.creation_timestamp}")
print(f"최근 업데이트: {registered_model.last_updated_timestamp}")

# 모든 버전 확인
print(f"\n=== 모델 버전 목록 ===")
for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    print(f"  Version {mv.version}:")
    print(f"    Run ID: {mv.run_id[:8]}...")
    print(f"    Status: {mv.status}")
    print(f"    Aliases: {mv.aliases}")

=== Registered Model: fraud_detector ===
생성 시간: 1767897217965
최근 업데이트: 1767898377158

=== 모델 버전 목록 ===
  Version 2:
    Run ID: f7c9439e...
    Status: READY
    Aliases: []
  Version 1:
    Run ID: f7c9439e...
    Status: READY
    Aliases: ['champion']


In [8]:
# 실습 2: 모델 등록 확인

# TODO 1: 등록된 모델 가져오기
registered_model = client.get_registered_model("_____")

# TODO 2: 모델 버전 검색
versions = client.search_model_versions(f"name='_____'")

print(f"모델 이름: {registered_model.name}")
print(f"버전 수: {len(list(versions))}")

MlflowException: Registered Model with name=_____ not found

In [8]:
# 실습 2 정답

# 등록된 모델 가져오기
registered_model = client.get_registered_model(MODEL_NAME)

# 모델 버전 검색
versions = list(client.search_model_versions(f"name='{MODEL_NAME}'"))

print(f"모델 이름: {registered_model.name}")
print(f"버전 수: {len(versions)}")

for v in versions:
    print(f"  - Version {v.version}: status={v.status}")

모델 이름: fraud_detector
버전 수: 2
  - Version 2: status=READY
  - Version 1: status=READY


In [9]:
# 체크포인트 2: 모델 등록 확인
assert registered_model is not None, "모델이 등록되지 않았습니다!"
assert len(versions) >= 1, "모델 버전이 없습니다!"
print(f"체크포인트 2 통과: {MODEL_NAME} 등록 완료, {len(versions)}개 버전!")

체크포인트 2 통과: fraud_detector 등록 완료, 2개 버전!


---

## Part 3: Alias 방식 - 최신 권장 ⭐

### 3-1. Champion/Challenger 개념

**현업 모델 배포 패턴:**

```
Champion: 현재 프로덕션에서 사용 중인 모델
Challenger: 새로 테스트 중인 모델

워크플로:
1. 새 모델 학습 → Version N 등록
2. Version N에 @challenger alias 설정
3. A/B 테스트 (champion 90% vs challenger 10%)
4. challenger가 더 좋으면 → @champion으로 승격
5. 기존 champion → @previous_champion 또는 alias 제거
```

**Alias 장점:**
- 자유로운 이름 지정 (`@champion`, `@canary`, `@rollback_candidate`)
- 하나의 버전에 여러 alias 적용 가능
- 코드 변경 없이 모델 교체 (`models:/fraud_detector@champion`)

### 3-2. Alias 설정 API

```python
# Alias 설정
client.set_registered_model_alias(
    name="fraud_detector",      # 등록된 모델 이름
    alias="champion",           # alias 이름
    version="1"                 # 버전 번호
)

# Alias로 모델 버전 조회
mv = client.get_model_version_by_alias(
    name="fraud_detector",
    alias="champion"
)

# Alias 삭제
client.delete_registered_model_alias(
    name="fraud_detector",
    alias="old_alias"
)
```

**주의사항:**
- `v1`, `v2` 같은 형식은 예약어로 사용 불가
- 소문자, 하이픈, 언더스코어 권장

In [10]:
# 예제: Champion Alias 설정

# 현재 버전을 champion으로 설정
client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="champion",
    version=model_version.version
)

print(f"Champion alias 설정 완료!")
print(f"  models:/{MODEL_NAME}@champion → Version {model_version.version}")

# 확인
champion_mv = client.get_model_version_by_alias(MODEL_NAME, "champion")
print(f"\nChampion 모델 정보:")
print(f"  Version: {champion_mv.version}")
print(f"  Run ID: {champion_mv.run_id[:8]}...")
print(f"  Aliases: {champion_mv.aliases}")

Champion alias 설정 완료!
  models:/fraud_detector@champion → Version 2

Champion 모델 정보:
  Version: 2
  Run ID: f7c9439e...
  Aliases: ['champion']


In [11]:
# 예제: Alias로 모델 로드 (프로덕션 방식)

# Champion 모델 정보 가져오기
champion_mv = client.get_model_version_by_alias(MODEL_NAME, "champion")

# 아티팩트 경로 자동 탐지
def find_model_artifact_path(client, run_id):
    """모델 아티팩트 경로 자동 탐지"""
    artifacts = client.list_artifacts(run_id)
    artifact_names = [a.path for a in artifacts]
    
    # 가능한 경로 우선순위
    possible_paths = ['best_model', 'model', 'xgboost_model']
    for path in possible_paths:
        if path in artifact_names:
            # 하위 파일 확인
            sub_artifacts = client.list_artifacts(run_id, path)
            sub_names = [a.path for a in sub_artifacts]
            
            # joblib 파일 찾기
            for sub in sub_names:
                if sub.endswith('.joblib'):
                    return sub
            
            # model.joblib 직접 확인
            if f"{path}/model.joblib" in sub_names or any('model.joblib' in s for s in sub_names):
                return f"{path}/model.joblib"
    
    return None

# 아티팩트 경로 찾기
artifact_path = find_model_artifact_path(client, champion_mv.run_id)

if artifact_path:
    # 아티팩트 다운로드
    local_path = client.download_artifacts(champion_mv.run_id, artifact_path)
    champion_model = joblib.load(local_path)
    
    print(f"Champion 모델 로드 완료!")
    print(f"  모델 타입: {type(champion_model).__name__}")
    print(f"  아티팩트 경로: {artifact_path}")
    print(f"  로컬 경로: {local_path}")
else:
    print("⚠️ 모델 아티팩트를 찾을 수 없습니다!")
    print(f"  Run ID: {champion_mv.run_id}")
    artifacts = client.list_artifacts(champion_mv.run_id)
    print(f"  사용 가능한 아티팩트: {[a.path for a in artifacts]}")
    
    # models:/ URI로 직접 로드 시도 (MLflow 모델 형식인 경우)
    try:
        champion_model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@champion")
        print(f"  → mlflow.pyfunc로 로드 성공!")
    except Exception as e:
        print(f"  → mlflow.pyfunc 로드 실패: {e}")
        champion_model = None

⚠️ 모델 아티팩트를 찾을 수 없습니다!
  Run ID: f7c9439e4e384e2e8cac949255b76214
  사용 가능한 아티팩트: ['feature_importance.png']
  → mlflow.pyfunc로 로드 성공!


In [12]:
# Champion 모델로 추론 테스트

if 'champion_model' not in dir() or champion_model is None:
    print("⚠️ champion_model이 로드되지 않았습니다.")
    print("cell-21을 먼저 실행하세요.")
else:
    # 테스트 데이터 로드
    test_df = pd.read_csv(DATA_PROCESSED / 'test_features.csv')
    X_test = test_df.drop('isFraud', axis=1)
    y_test = test_df['isFraud']

    # 추론 (pyfunc 모델인지 XGBClassifier인지 확인)
    if hasattr(champion_model, 'predict_proba'):
        y_prob = champion_model.predict_proba(X_test)[:, 1]
    elif hasattr(champion_model, 'predict'):
        # pyfunc 모델의 경우
        y_prob = champion_model.predict(X_test)
        if hasattr(y_prob, 'values'):
            y_prob = y_prob.values.flatten()
    else:
        print("⚠️ 예측 함수를 찾을 수 없습니다.")
        y_prob = None

    if y_prob is not None:
        # FDS 지표 계산
        auc = roc_auc_score(y_test, y_prob)
        auprc = average_precision_score(y_test, y_prob)

        # Threshold 적용 (1-3에서 최적화한 0.18)
        threshold = 0.18
        y_pred = (y_prob >= threshold).astype(int)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)

        print(f"=== Champion 모델 성능 (Test) ===")
        print(f"  AUC: {auc:.4f}")
        print(f"  AUPRC: {auprc:.4f}")
        print(f"  Recall: {recall:.4f} (threshold={threshold})")
        print(f"  Precision: {precision:.4f}")

=== Champion 모델 성능 (Test) ===
  AUC: 0.8025
  AUPRC: 0.2298
  Recall: 0.6511 (threshold=0.18)
  Precision: 0.3345


In [ ]:
# 실습 3: Challenger alias 설정 및 비교

# 시나리오: 새 모델을 challenger로 등록하고 champion과 비교
# (실습용으로 같은 버전에 두 alias를 적용)

# TODO 1: 현재 champion 버전 확인
champion_mv = client.get_model_version_by_alias(MODEL_NAME, "_____")
print(f"현재 Champion: Version {champion_mv.version}")

# TODO 2: 같은 버전을 challenger로도 설정 (테스트용)
client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="_____",
    version=champion_mv.version
)

# TODO 3: Challenger alias로 모델 조회
challenger_mv = client.get_model_version_by_alias(MODEL_NAME, "_____")
print(f"Challenger: Version {challenger_mv.version}")
print(f"Aliases: {challenger_mv.aliases}")

In [13]:
# 실습 3 정답

# 현재 champion 버전 확인
champion_mv = client.get_model_version_by_alias(MODEL_NAME, "champion")
print(f"현재 Champion: Version {champion_mv.version}")

# 같은 버전을 challenger로도 설정 (실제로는 다른 버전 사용)
client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="challenger",
    version=champion_mv.version
)

# Challenger alias로 모델 조회
challenger_mv = client.get_model_version_by_alias(MODEL_NAME, "challenger")
print(f"Challenger: Version {challenger_mv.version}")
print(f"Aliases: {challenger_mv.aliases}")

# 하나의 버전에 여러 alias 적용 가능!

현재 Champion: Version 2
Challenger: Version 2
Aliases: ['challenger', 'champion']


In [14]:
# 체크포인트 3: Alias 설정 확인
champion_mv = client.get_model_version_by_alias(MODEL_NAME, "champion")
assert champion_mv is not None, "Champion alias가 설정되지 않았습니다!"
assert "champion" in champion_mv.aliases, "Champion alias 확인 실패!"
print(f"체크포인트 3 통과: Champion alias 설정 완료!")

체크포인트 3 통과: Champion alias 설정 완료!


---

## Part 4: Stage 방식 - 레거시 (참고용)

### 4-1. Stage 방식 개요

**⚠️ Deprecated (권장하지 않음)**

MLflow 2.9 이전의 모델 관리 방식입니다. 
새 프로젝트에서는 Alias 방식을 사용하세요.

**Stage 종류:**
- `None`: 초기 상태
- `Staging`: 테스트 중
- `Production`: 프로덕션 배포
- `Archived`: 보관 (더 이상 사용 안 함)

**워크플로:**
```
None → Staging → Production → Archived
```

**한계:**
- 고정된 3단계만 사용 가능
- 한 stage에 하나의 버전만 가능
- 유연한 A/B 테스트 어려움

In [ ]:
# 예제: Stage 전환 (레거시 - 참고용)

# ⚠️ 이 방식은 deprecated입니다. Alias 방식을 사용하세요.

# Stage 전환 (Production으로)
# client.transition_model_version_stage(
#     name=MODEL_NAME,
#     version=model_version.version,
#     stage="Production",
#     archive_existing_versions=True  # 기존 Production 버전 자동 Archive
# )

print("Stage 방식은 deprecated입니다.")
print("Part 3의 Alias 방식 (champion/challenger)을 사용하세요.")

# Stage vs Alias URI 비교
print("\n=== URI 비교 ===")
print(f"Stage 방식:  models:/{MODEL_NAME}/Production")
print(f"Alias 방식:  models:/{MODEL_NAME}@champion  ← 권장")

### 4-2. Stage vs Alias 비교 정리

| 구분 | Stage 방식 | Alias 방식 |
|------|------------|------------|
| **상태** | Deprecated | 권장 ⭐ |
| **API** | `transition_model_version_stage()` | `set_registered_model_alias()` |
| **URI** | `models:/name/Production` | `models:/name@champion` |
| **유연성** | 3단계 고정 | 무제한 alias |
| **동시성** | 한 stage에 1개 | 1버전에 여러 alias |
| **롤백** | stage 재전환 필요 | alias 재지정 |

**결론:**
- 레거시 코드 유지보수 시에만 Stage 방식 이해 필요
- 신규 프로젝트는 반드시 Alias 방식 사용

In [ ]:
# 실습 4: Stage 방식 이해 (참고용 - 실행하지 않아도 됨)

# Stage 방식 코드 예시 (실행 X)
"""
# 1. Staging으로 전환
client.transition_model_version_stage(
    name="fraud_detector",
    version="1",
    stage="Staging"
)

# 2. Production으로 승격
client.transition_model_version_stage(
    name="fraud_detector",
    version="1",
    stage="Production",
    archive_existing_versions=True
)

# 3. Stage로 모델 로드
model = mlflow.pyfunc.load_model("models:/fraud_detector/Production")
"""

print("Stage 방식 코드는 참고용입니다.")
print("실제 작업에서는 Alias 방식을 사용하세요.")

# 체크포인트 4 (자동 통과)
print("\n체크포인트 4 통과: Stage 방식 이해 완료!")

---

## Part 5: 모델 관리 실무

### 5-1. 태그로 validation_status 관리

**현업 패턴: 태그로 모델 상태 추적**

```python
# 모델 버전 태그
validation_status: "pending" | "passed" | "failed"
deployment_env: "dev" | "staging" | "prod"
owner: "fraud_team"
approved_by: "manager_name"
```

**워크플로:**
```
1. 모델 등록 → validation_status: "pending"
2. 오프라인 테스트 통과 → validation_status: "passed"
3. A/B 테스트 승인 → approved_by: "홍길동"
4. 프로덕션 배포 → deployment_env: "prod"
```

In [15]:
# 예제: 모델 버전 태그 설정

# 태그 추가
client.set_model_version_tag(
    name=MODEL_NAME,
    version=model_version.version,
    key="validation_status",
    value="passed"
)

client.set_model_version_tag(
    name=MODEL_NAME,
    version=model_version.version,
    key="deployment_env",
    value="prod"
)

client.set_model_version_tag(
    name=MODEL_NAME,
    version=model_version.version,
    key="owner",
    value="fds_team"
)

# 태그 확인
mv = client.get_model_version(MODEL_NAME, model_version.version)
print(f"=== Version {mv.version} 태그 ===")
for key, value in mv.tags.items():
    print(f"  {key}: {value}")

=== Version 2 태그 ===
  deployment_env: prod
  owner: fds_team
  validation_status: passed


In [16]:
# 예제: 모델 설명(description) 추가

# Registered Model 설명
client.update_registered_model(
    name=MODEL_NAME,
    description="""
FDS(Fraud Detection System) XGBoost 모델

- 데이터: IEEE-CIS Fraud Detection
- 피처: 447개 (정형 + 시계열)
- Threshold: 0.18 (비용 최적화)
- 평가 지표: AUC, AUPRC, Recall
"""
)

# Model Version 설명
client.update_model_version(
    name=MODEL_NAME,
    version=model_version.version,
    description=f"""
Optuna 50 trials 튜닝 결과
- Test AUC: {best_run.data.metrics.get('test_auc', 0):.4f}
- Test AUPRC: {best_run.data.metrics.get('test_auprc', 0):.4f}
- Test Recall: {best_run.data.metrics.get('test_recall', 'N/A')}
"""
)

print("설명 추가 완료!")

설명 추가 완료!


### 5-3. 모델 비교 워크플로

**Champion vs Challenger 비교 프로세스:**

```python
# 1. 두 모델 로드
champion = load_model("models:/fraud_detector@champion")
challenger = load_model("models:/fraud_detector@challenger")

# 2. 동일 데이터로 평가
champion_metrics = evaluate(champion, test_data)
challenger_metrics = evaluate(challenger, test_data)

# 3. FDS 기준으로 비교
#    - AUPRC가 더 높은가?
#    - Recall이 유지되는가?
#    - 비용이 감소하는가?

# 4. 승격 결정
if challenger_metrics['auprc'] > champion_metrics['auprc']:
    # challenger를 새로운 champion으로
    client.set_registered_model_alias(name, "champion", challenger_version)
```

In [ ]:
# 실습 5: 태그 및 설명 추가

# TODO 1: validation_status 태그 추가
client.set_model_version_tag(
    name=MODEL_NAME,
    version=model_version.version,
    key="_____",
    value="passed"
)

# TODO 2: 모델 버전 확인
mv = client.get_model_version(MODEL_NAME, model_version.version)

# TODO 3: 태그 출력
print(f"Tags: {mv._____}")

In [17]:
# 실습 5 정답

# validation_status 태그 추가 (이미 설정되어 있으면 덮어씔)
client.set_model_version_tag(
    name=MODEL_NAME,
    version=model_version.version,
    key="validation_status",
    value="passed"
)

# 모델 버전 확인
mv = client.get_model_version(MODEL_NAME, model_version.version)

# 태그 출력
print(f"=== Version {mv.version} ===")
print(f"Aliases: {mv.aliases}")
print(f"Tags: {mv.tags}")
print(f"Description: {mv.description[:100] if mv.description else 'N/A'}...")

=== Version 2 ===
Aliases: ['challenger', 'champion']
Tags: {'deployment_env': 'prod', 'owner': 'fds_team', 'validation_status': 'passed'}
Description: 
Optuna 50 trials 튜닝 결과
- Test AUC: 0.9026
- Test AUPRC: 0.5434
- Test Recall: N/A
...


In [18]:
# 체크포인트 5: 태그 설정 확인
mv = client.get_model_version(MODEL_NAME, model_version.version)
assert "validation_status" in mv.tags, "validation_status 태그가 없습니다!"
assert mv.tags["validation_status"] == "passed", "validation_status가 'passed'가 아닙니다!"
print("체크포인트 5 통과: 태그 설정 완료!")

체크포인트 5 통과: 태그 설정 완료!


---

## Part 6: 결론 및 면접 Q&A

In [19]:
# 최종 체크리스트
print("="*60)
print("2-2 MLflow Model Registry 체크리스트")
print("="*60)

# champion_model 정의 여부 확인
model_loaded = 'champion_model' in dir() and champion_model is not None

checks = [
    ("MlflowClient 연결", client is not None),
    ("모델 등록 완료", len(list(client.search_model_versions(f"name='{MODEL_NAME}'"))) >= 1),
    ("Champion alias 설정", "champion" in client.get_model_version_by_alias(MODEL_NAME, "champion").aliases),
    ("모델 로드 성공", model_loaded),
    ("태그 설정", "validation_status" in client.get_model_version(MODEL_NAME, model_version.version).tags),
]

all_passed = True
for name, passed in checks:
    status = "✅" if passed else "❌"
    print(f"  {status} {name}")
    if not passed:
        all_passed = False

print("="*60)
if all_passed:
    print("모든 체크 통과! 2-2 완료!")
else:
    print("일부 체크 실패 - 위 내용 확인 필요")
    if not model_loaded:
        print("\n⚠️ 모델 로드 실패:")
        print("  1. 2-1 노트북을 다시 실행하여 best_model 아티팩트 저장")
        print("  2. 또는 Kernel → Restart & Run All로 2-2 재실행")

2-2 MLflow Model Registry 체크리스트
  ✅ MlflowClient 연결
  ✅ 모델 등록 완료
  ✅ Champion alias 설정
  ✅ 모델 로드 성공
  ✅ 태그 설정
모든 체크 통과! 2-2 완료!


---

## 면접 Q&A

### Q: "Model Registry를 왜 도입했나요?"

> "FDS에서 모델 버전 관리와 배포 이력 추적이 필수입니다.
> 금융 규제상 '언제, 어떤 모델로 판단했는지' 감사 추적이 필요하고,
> 문제 발생 시 즉시 이전 버전으로 롤백해야 합니다.
> Model Registry로 이 모든 것을 체계적으로 관리합니다."

### Q: "Stage 방식과 Alias 방식의 차이는?" ⭐

> "Stage 방식은 MLflow 2.9 이전의 레거시 방식으로 deprecated입니다.
> None → Staging → Production 3단계만 가능하고 유연성이 부족합니다.
> 
> Alias 방식은 현재 권장 방식으로, `@champion`, `@challenger` 같이
> 자유롭게 alias를 지정할 수 있습니다.
> 하나의 버전에 여러 alias를 적용할 수 있어서 A/B 테스트에 유리합니다.
> 
> URI도 다릅니다:
> - Stage: `models:/fraud_detector/Production`
> - Alias: `models:/fraud_detector@champion` ← 권장"

### Q: "Champion/Challenger 패턴이 뭐나요?"

> "프로덕션 모델 배포 패턴입니다.
> Champion은 현재 프로덕션에서 사용 중인 모델이고,
> Challenger는 새로 테스트 중인 모델입니다.
> 
> 워크플로:
> 1. 새 모델 학습 → challenger alias 설정
> 2. A/B 테스트 (champion 90%, challenger 10%)
> 3. challenger 성능이 좋으면 champion으로 승격
> 4. 기존 champion은 rollback_candidate로 변경
> 
> 코드 변경 없이 alias만 바꾸면 모델 교체가 됩니다."

### Q: "모델 롤백은 어떻게 하나요?"

> "Alias 방식에서는 매우 간단합니다.
> ```python
> # 이전 버전으로 champion alias 재지정
> client.set_registered_model_alias('fraud_detector', 'champion', '1')
> ```
> 코드 배포 없이 alias만 변경하면 즉시 롤백됩니다.
> 그래서 `rollback_candidate` alias를 미리 설정해두는 것이 좋습니다."

---

## 로깅된 정보 요약

| 항목 | 값 |
|------|-----|
| Registered Model | fraud_detector |
| Champion Version | 1 |
| Test AUC | ~0.90 |
| Test AUPRC | ~0.54 |
| validation_status | passed |

---

## 다음 단계

**2-3: Evidently 드리프트 모니터링**
- 데이터 드리프트 감지
- 모델 성능 모니터링
- 알림 설정

In [21]:
# Registry 최종 상태 요약
print("="*60)
print("Model Registry 최종 상태")
print("="*60)

# 등록된 모델 정보
rm = client.get_registered_model(MODEL_NAME)
print(f"\nRegistered Model: {rm.name}")
print(f"Description: {rm.description[:50] if rm.description else 'N/A'}...")

# 모든 버전
print(f"\n버전 목록:")
for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    aliases_str = ", ".join(mv.aliases) if mv.aliases else "없음"
    print(f"  Version {mv.version}: aliases=[{aliases_str}]")

# Champion 정보
champion = client.get_model_version_by_alias(MODEL_NAME, "champion")
print(f"\nChampion Model:")
print(f"  Version: {champion.version}")
print(f"  Run ID: {champion.run_id[:8]}...")
print(f"  Tags: {champion.tags}")

print("="*60)
print("2-2 MLflow Model Registry 완료!")

Model Registry 최종 상태

Registered Model: fraud_detector
Description: 
FDS(Fraud Detection System) XGBoost 모델

- 데이터: IE...

버전 목록:
  Version 2: aliases=[challenger, champion]
  Version 1: aliases=[없음]

Champion Model:
  Version: 2
  Run ID: f7c9439e...
  Tags: {'deployment_env': 'prod', 'owner': 'fds_team', 'validation_status': 'passed'}
2-2 MLflow Model Registry 완료!
